In [25]:
import os
HOME = os.getcwd()
print('HOME: ',HOME)

import math
import json
import random
import pandas as pd
import sys
import lxml
import sklearn as sk
import numpy as np

from sklearn.manifold import TSNE
from sklearn.cluster import KMeans
from sklearn.cluster import HDBSCAN, DBSCAN
from scipy.spatial.distance import cosine
from sklearn.metrics.pairwise import cosine_similarity

import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import plotly.graph_objects as go

from scipy.sparse import csr_matrix, issparse


HOME:  /Users/ericsuardi/Desktop/DataMiningProject23-24


In [26]:
# STANDARD_FILE = 'standard_small_order_printClustroids.json'
# ACTUAL_FILE = 'actual_small_order_printClustroids.json'

# STANDARD_FILE = 'standard_small.json'
# ACTUAL_FILE = 'actual_small.json'

STANDARD_FILE = 'standard_medium_new.json'
ACTUAL_FILE = 'actual_medium_new.json'

# STANDARD_FILE = 'standard_big.json'
# ACTUAL_FILE = 'actual_big.json'

K_SHINGLES = 3

In [27]:
# load standard and actual data
with open(os.path.join('data',STANDARD_FILE)) as f:
    standard = json.load(f)

with open(os.path.join('data', ACTUAL_FILE)) as f:
    actual = json.load(f)

# load the data into a dataframe
dfStandard = pd.DataFrame(standard)
dfActual = pd.DataFrame(actual)

# print head of the dataframes
print(dfStandard.head())
print(dfActual.head())

# get the unique cities and items of the standard data
cities = []
items = []
longestRoute = 0
shortestRoute = np.inf
maxItemQuantity = 0

standardRefIds = []
for index, s in dfStandard.iterrows():
    #print(s)
    idS = s['id']
    route = s['route']
    standardRefIds.append(int(idS[1]))
    for trip in route:
        cities.append(trip['from']) 
        items.extend(trip['merchandise'].keys())
        maxItemQuantity = max(maxItemQuantity, max(trip['merchandise'].values()))
    if len(route) > 0:
        cities.append(route[-1]['to'])
        
    if len(route) > longestRoute:
        longestRoute = len(route)
        
    if len(route) < shortestRoute:
        shortestRoute = len(route)

actualRefStandardIds = []
for index, s in dfActual.iterrows():
    #print(s)
    idS = s['id']
    route = s['route']
    idStandard = s['sroute']
    actualRefStandardIds.append(int(idStandard[1]))
    for trip in route:
        cities.append(trip['from'])
        items.extend(trip['merchandise'].keys())
        maxItemQuantity = max(maxItemQuantity, max(trip['merchandise'].values()))
        
    if len(route) > 0:
        cities.append(route[-1]['to'])
        
    if len(route) > longestRoute:
        longestRoute = len(route)
    
    if len(route) < shortestRoute:
        shortestRoute = len(route)

# find the unique cities and items
uniqueCities = sorted(list(set(cities)))
#uniqueCities.insert(0, 'NULL')          # add NULL city, for padding vectors with different lengths (trips in routes)
uniqueItems = sorted(list(set(items)))

if shortestRoute < 2:
    K_SHINGLES = 2

threeShingles = []

for i, c1 in enumerate(uniqueCities):
    for j, c2 in enumerate(uniqueCities):
        if i == j:
            continue
        for k, c3 in enumerate(uniqueCities):
            if j == k or i == k:
                continue
            threeShingles.append([c1, c2, c3])
            
permutations = math.perm(len(uniqueCities), K_SHINGLES)

print("\nUnique cities: ", uniqueCities)
print("Unique items: ", uniqueItems)

print("\nNumber of cities: ", len(uniqueCities))
print("Number of items: ", len(uniqueItems))

print("\nLongest route: ", longestRoute)
print("Shortest route: ", shortestRoute)

print("\nMax item quantity: ", maxItemQuantity)

print("\nNumber of three-shingles: ", len(threeShingles))

print(f"\n{K_SHINGLES}-shingles: ", math.perm(len(uniqueCities), K_SHINGLES))
print(f"{K_SHINGLES}-shingles: ", math.comb(len(uniqueCities), K_SHINGLES))

print(f"\n\033[92mK-Shingles used: {K_SHINGLES} \033[0m")


   id                                              route
0  s0  [{'from': 'Teramo', 'to': 'Varese', 'merchandi...
1  s1  [{'from': 'Trapani', 'to': 'Carrara', 'merchan...
2  s2  [{'from': 'Sanremo', 'to': 'Carrara', 'merchan...
3  s3  [{'from': 'Tivoli', 'to': 'Legnano', 'merchand...
4  s4  [{'from': 'Carrara', 'to': 'Bagheria', 'mercha...
   id driver sroute                                              route
0  a0    E_3     s0  [{'from': 'Teramo', 'to': 'Varese', 'merchandi...
1  a1    M_2     s0  [{'from': 'Teramo', 'to': 'Varese', 'merchandi...
2  a2    X_1     s0  [{'from': 'Teramo', 'to': 'Varese', 'merchandi...
3  a3    C_0     s0  [{'from': 'Varese', 'to': 'Sanremo', 'merchand...
4  a4    Z_2     s0  [{'from': 'Teramo', 'to': 'Varese', 'merchandi...

Unique cities:  ['Aprilia', 'Bagheria', 'Barletta', 'Bergamo', 'Bitonto', 'Carrara', 'Caserta', 'Castellammare di Stabia', 'Catania', 'Florence', 'Foligno', 'Forlì', 'Gela', 'Legnano', 'Livorno', 'Manfredonia', 'Modica', 'Molfetta'

In [28]:
def hashShingles(shingles, n):
    # hash shingles
    string = "" 
    for shingle in shingles:
        string += str(shingle) + "," # [45, 4, 8] -> "45,4,8,"
    
    return hash(string) #% n

def createShingles(df, k, uniqueCities, uniqueItems, longestRoute, maxItemQuantity, permutations):
    # create shingles for each route
    shingles = []
    for index, s in df.iterrows():
        idS = s['id']
        route = s['route']
        shingle = [index]
        citiesInRoute = [] # napoli roma milano teramo bergamo [10,4,5,48,12] [10,4,5] [4,5,48] [5,48,12]
        merchandiseInRoute = np.zeros(len(uniqueItems))
        for trip in route:
            citiesInRoute.append(uniqueCities.index(trip['from']))
            #merchandiseInRoute += np.array(list(trip['merchandise'].values()))
            for item, n in trip['merchandise'].items():
                merchandiseInRoute[uniqueItems.index(item)] += n
        if len(route) > 0:
            citiesInRoute.append(uniqueCities.index(route[-1]['to']))
        if len(route) > 0:
            merchandiseInRoute = merchandiseInRoute / (maxItemQuantity*len(route))
        
        hashedShingles = []
        for i in range(len(citiesInRoute)-k+1):
            # Q: is it correct to set the modulo for the hash function to the number of permutations?
            # A: yes, because we want to have a unique hash for each shingle
            # Q: would it be better to use a different hash function?
            # A: yes, because the modulo function is not a good hash function
            hashedShingles.append(hashShingles(citiesInRoute[i:i+k], permutations) )
        
        shingle.append(np.array(hashedShingles))
        
        shingle.append(merchandiseInRoute) # quantity hot encoding
        
        shingles.append(shingle)
        
    return shingles # [ index, [shingles], [merchandise] ]

In [29]:
standardSets = createShingles(dfStandard, k=K_SHINGLES, uniqueCities=uniqueCities, uniqueItems=uniqueItems, longestRoute=longestRoute, maxItemQuantity=maxItemQuantity, permutations=permutations)
actualSets = createShingles(dfActual, k=K_SHINGLES, uniqueCities=uniqueCities, uniqueItems=uniqueItems, longestRoute=longestRoute, maxItemQuantity=maxItemQuantity, permutations=permutations)

print("\nstandardSets", len(standardSets), "shape first element", standardSets[0][1].shape, standardSets[0])
print("\nactualSets", len(actualSets),  "shape first element", standardSets[0][1].shape, actualSets[0])

print("\nstandardSets:", len(standardSets))
print("actualSets:", len(actualSets))

assert len(standardSets[0]) == 3, "The length of the standard set is not equal to 3 (index, shingles, merchandise)"
assert len(standardSets[0][2]) == len(uniqueItems), "The length of the merchandise vector is not equal to the number of unique items"


standardSets 10 shape first element (24,) [0, array([ 4402244120563585138,  3531698521109399935, -1261429527285150229,
       -5902054544079464567,   720682959376929508, -7317660469128219896,
       -6922549371521355299, -6955868140266794909, -6737381728946263322,
       -7307798614190910509,  2465406913129648046, -2073312785587459803,
         720723470017730132,   610244005815212293,  8285260583327599795,
        3300074537833087242, -1725791906546253351,  1334132584688507166,
       -6258754605945830738,  2657487059942561339,  1891624280221708430,
       -3938129540339955320, -2139331395444106252,  5822030145497066879]), array([0.25541667, 0.29333333, 0.23166667, 0.28458333, 0.17416667,
       0.26916667, 0.34125   , 0.25833333, 0.32125   , 0.28291667])]

actualSets 10000 shape first element (24,) [0, array([ 4402244120563585138,  3531698521109399935, -1261429527285150229,
       -5902054544079464567,   720682959376929508, -7737137696242698271,
        3929503038652343720, -6922549

## Clustering

In [30]:


def jaccard_similarity_matrix(matrix):
    if issparse(matrix):
        print("matrix is sparse")
        
        matrixCSR = csr_matrix(matrix)
        intersection = np.dot(matrixCSR, matrixCSR.T)
        intersection = intersection.todense()
        print("intersection", intersection.shape, type(intersection))
        #print("intersection", intersection.toarray(), type(intersection.toarray()))
        row_sums = matrix.sum(axis=1)
        print("row_sums", row_sums.shape)
        union = row_sums[:, None] + row_sums - intersection
        print("union", union.shape)
        union = np.where(union == 0, 1, union)  # avoid division by zero
        print("union", union.shape)
        jaccard_similarity = intersection / union
        print("jaccard_similarity", jaccard_similarity.shape, type(jaccard_similarity))
    else:
        print("matrix is not sparse")
        
        intersection = np.dot(matrix, matrix.T)
        print("intersection", intersection.shape)
        #print("intersection", intersection.toarray(), type(intersection.toarray()))
        row_sums = matrix.sum(axis=1)
        print("row_sums", row_sums.shape)
        union = row_sums[:, None] + row_sums - intersection
        print("union", union.shape)
        union = np.where(union == 0, 1, union)
        jaccard_similarity = intersection / union
        print("jaccard_similarity", jaccard_similarity.shape)
    print("jaccard_similarity contains nan", np.isnan(jaccard_similarity).any())   
    return jaccard_similarity

# def jaccard_similarity_matrix(matrix):
#     print("matrix", matrix.shape)
#     intersection = np.dot(matrix, matrix.T)
#     #print("intersection", intersection.shape)
#     #print("intersection", intersection.toarray(), type(intersection.toarray()))
#     row_sums = matrix.sum(axis=1)
#     #print("row_sums", row_sums.shape)
#     union = row_sums[:, None] + row_sums - intersection
#     #print("union", union.shape)
#     union = np.where(union == 0, 1, union)  # avoid division by zero
#     #print("union", union.shape)
#     jaccard_similarity = intersection / union
#     #print("jaccard_similarity", jaccard_similarity.shape)
#     return jaccard_similarity

# def jaccard_similarity_matrix(matrix):
#     matrixCSR = scipy.sparse.csr_matrix(matrix)
#     intersection = np.dot(matrixCSR, matrixCSR.T)
#     print("intersection", intersection.shape)
#     row_sums = matrixCSR.sum(axis=1)
#     print("row_sums", row_sums.shape)
#     union = row_sums[:, None] + row_sums - intersection
#     print("union", union.shape)
#     union = np.where(union == 0, 1, union)  # avoid division by zero
#     print("union", union.shape)
#     jaccard_similarity = intersection / union
#     return np.array(jaccard_similarity.toarray())



# def jaccard_similarity_matrix(matrix):
#     # Compute dot product for upper triangular part of the matrix
#     upper_triangular_indices = np.triu_indices(matrix.shape[0], k=1)
#     print("upper_triangular_indices", upper_triangular_indices[0].shape, upper_triangular_indices[1].shape, upper_triangular_indices[0])
#     print(matrix[upper_triangular_indices[0]])
#     intersection = np.dot(matrix[upper_triangular_indices[0]], matrix[upper_triangular_indices[1]].T) # 0, 1, 2 

#     row_sums = matrix.sum(axis=1)
#     union = row_sums[:, None] + row_sums - intersection

#     # Avoid division by zero
#     union = np.where(union == 0, 1, union)

#     jaccard_similarity = intersection / union

#     # Create a full symmetric matrix from the upper triangular part
#     jaccard_similarity_full = np.zeros((matrix.shape[0], matrix.shape[0]))
#     jaccard_similarity_full[upper_triangular_indices] = jaccard_similarity
#     jaccard_similarity_full.T[upper_triangular_indices] = jaccard_similarity

#     return jaccard_similarity_full

# def jaccard_similarity_matrix(matrix):
#     n = matrix.shape[0]
#     intersection = np.zeros((n, n))
#     for i in range(n):
#         for j in range(i+1, n):
#             intersection[i, j] = np.dot(matrix[i], matrix[j])
#     intersection += intersection.T
#     print("intersection", intersection.shape)
#     row_sums = matrix.sum(axis=1)
#     print("row_sums", row_sums.shape)
#     union = row_sums[:, None] + row_sums - intersection
#     print("union", union.shape)
#     union = np.where(union == 0, 1, union)  # avoid division by zero
#     print("union", union.shape)
#     jaccard_similarity = intersection / union
#     return jaccard_similarity

# def jaccard_similarity_matrix(matrix):
#     row_sums = matrix.sum(axis=1)
#     intersection = np.einsum('ij,ij->i', matrix, matrix.T)
#     union = row_sums[:, None] + row_sums - intersection
#     union = np.where(union == 0, 1, union)  # avoid division by zero
#     jaccard_similarity = intersection / union
#     return jaccard_similarity

def jaccard_similarity_two_matrices(matrix1, matrix2):
    #intersection = np.dot(matrix, matrix.T)
    intersection = np.dot(matrix1, matrix2.T)
    row_sums1 = matrix1.sum(axis=1)
    row_sums2 = matrix2.sum(axis=1)
    union = row_sums1[:, None] + row_sums2 - intersection
    union = np.where(union == 0, 1, union)  # avoid division by zero
    jaccard_similarity = intersection / union
    return jaccard_similarity

# def jaccard_similarity_matrix_merch(matrix):
#     # get the minimum matrix (square since compares every pair), where i,j is the sum of the minimum values of i and j
#     print("matrix", matrix.shape)
#     min_matrix = np.minimum.outer(matrix, matrix) # 4d matrix (i,j,k,l) where i,j are the rows and columns of the matrix and k,l are the elements of the matrix
#     print("min_matrix", min_matrix.shape)
#     # now sum the minimum matrix along the last two dimensions (k,l) bec
#     sum_min_matrix = np.sum(min_matrix, axis=(1,3)) # 3d matrix (i,j,k) where i,j are the rows and columns of the matrix and k is the sum of the minimum values of i and j
#     #sum_min_matrix = np.sum(sum_min_matrix, axis=-1) # 2d matrix (i,j) where i,j are the rows and columns of the matrix and k is the sum of the minimum values of i and j
#     print("sum_min_matrix", sum_min_matrix.shape)
#     # get the union matrix (square since compares every pair), where i,j is the sum of the maximum values of i and j
#     max_matrix = np.maximum.outer(matrix, matrix)
#     print("max_matrix", max_matrix.shape)
#     # now sum the maximum matrix along the last two dimensions (k,l) bec
#     sum_max_matrix = np.sum(max_matrix, axis=(1,3))
#     #sum_max_matrix = np.sum(sum_max_matrix, axis=-1)
#     print("sum_max_matrix", sum_max_matrix.shape)
#     # divide the sum of the minimum matrix by the sum of the maximum matrix
#     jaccard_similarity = sum_min_matrix / sum_max_matrix
#     return jaccard_similarity

def jaccard_similarity_matrix_merch(matrix):
    print("matrix", matrix.shape)
    min_matrix = np.minimum(matrix[:, None, :], matrix[None, :, :]) # (10, 100) -> (10, 1, 100) -> (1, 10, 100) -> (10, 10, 100)
    sum_min_matrix = np.sum(min_matrix, axis=-1)
    print("sum_min_matrix", sum_min_matrix.shape)
    
    max_matrix = np.maximum(matrix[:, None, :], matrix[None, :, :])
    sum_max_matrix = np.sum(max_matrix, axis=-1)
    print("sum_max_matrix", sum_max_matrix.shape)
    
    jaccard_similarity = sum_min_matrix / sum_max_matrix
    return jaccard_similarity

# def jaccard_similarity_matrix_merch(matrix): # [0,0.3,0,0.1,0,0] lungo uniqueItems
#     print("matrix", matrix.shape)
#     print("is sparse", issparse(matrix))
#     # Convert the input matrix to a CSR matrix
#     matrix_csr = csr_matrix(matrix)
#     print("matrix_csr", matrix_csr.shape)
#     shape_min = (matrix_csr.shape[0], 1, matrix_csr.shape[1])
#     shape_min2 = (1, matrix_csr.shape[0], matrix_csr.shape[1])
#     print("shape_min", shape_min)
#     print("shape_min", shape_min2)
#     # create a 3d matrix where i want 10 copies of the matrix 
#     matrix_csr1 = np.stack([matrix_csr for _ in range(1000)], axis=0)
#     print("matrix_csr1", matrix_csr1.shape)
#     print("matrix_csr reshaped", matrix_csr)
#     # Compute the minimum matrix and its sum
#     # min_matrix = matrix_csr.minimum(matrix_csr.T)  # (1000, 10)  10, 1000 -> 1000, 1000
    
#     # sum_min_matrix = min_matrix.sum(axis=-1).A.flatten()

#     # print("sum_min_matrix", sum_min_matrix.shape)

#     # # Compute the maximum matrix and its sum
#     # max_matrix = matrix_csr.maximum(matrix_csr.T)
#     # sum_max_matrix = max_matrix.sum(axis=-1).A.flatten()

#     # print("sum_max_matrix", sum_max_matrix.shape)

#     # # Compute Jaccard similarity
#     # jaccard_similarity = sum_min_matrix / sum_max_matrix
#     # return jaccard_similarity
#     return 1

    # current wrong method
    # 1 2 3    1 4 7    1 2 3  6     6 6 6
    # 4 5 6    2 5 8    2 5 6  13    6 15 15
    # 7 8 9    3 6 9    3 6 9        6 15 24
    
    
    # right method
    # 1 2 3    1 2 3    1 2 3
    # 1 2 3    4 5 6    1 2 3
    # 1 2 3    7 8 9    1 2 3
    
    # 4 5 6    1 2 3    1 2 3
    # 4 5 6    4 5 6    4 5 6
    # 4 5 6    7 8 9    4 5 6
    
    # 7 8 9    1 2 3    1 2 3
    # 7 8 9    4 5 6    4 5 6
    # 7 8 9    7 8 9    7 8 9
    
    
    #                   6 6 6
    #                   6 15 15
    #                   6 15 24
    
def similarity_matrix_merch(matrix):
    if issparse(matrix):
        print("merch matrix is sparse")
        matrix = csr_matrix(matrix)
    print("matrix merch shape", matrix.shape)
    simMatrix = cosine_similarity(matrix)
    
    return simMatrix

# def create_binary_matrix(routeSets):
#     # create binary matrix where each row represents a route
#     uniqueShingles = list(set([shingle for route in routeSets for shingle in route[1]]))
#     print("uniqueShingles", len(uniqueShingles))
#     binaryMatrix = np.zeros((len(routeSets), len(uniqueShingles)))
#     for i, route in enumerate(routeSets):
#         for shingle in route[1]:
#             binaryMatrix[i][uniqueShingles.index(shingle)] = 1
#     return binaryMatrix

def create_binary_matrix(routeSets):
    uniqueShingles = list(set(shingle for route in routeSets for shingle in route[1]))
    print("uniqueShingles", len(uniqueShingles))

    # Create a dictionary that maps each shingle to its index
    shingle_to_index = {shingle: index for index, shingle in enumerate(uniqueShingles)}
    print("shingle_to_index", len(shingle_to_index))

    binaryMatrix = np.zeros((len(routeSets), len(uniqueShingles)), dtype=int)

    for i, route in enumerate(routeSets):
        #print("i", i)
        # Get the indices of the shingles in this route
        indices = [shingle_to_index[shingle] for shingle in route[1]]
        # Use advanced indexing to set the corresponding elements in the binary matrix to 1
        binaryMatrix[i, indices] = 1

    return binaryMatrix

def create_binary_matrices(routeSet1, routeSet2):
    # create binary matrix where each row represents a route
    uniqueShinglesBoth = list(set([shingle for route in routeSet1 for shingle in route[1]] + [shingle for route in routeSet2 for shingle in route[1]]))
    binaryMatrix1 = np.zeros((len(routeSet1), len(uniqueShinglesBoth)))
    binaryMatrix2 = np.zeros((len(routeSet2), len(uniqueShinglesBoth)))
    for i, route in enumerate(routeSet1):
        for shingle in route[1]:
            binaryMatrix1[i][uniqueShinglesBoth.index(shingle)] = 1
            
    for i, route in enumerate(routeSet2):
        for shingle in route[1]:
            binaryMatrix2[i][uniqueShinglesBoth.index(shingle)] = 1
    return binaryMatrix1, binaryMatrix2

# convert routes and merchandise to binary matrices
# binary matrix where each row represents a route
route_matrix = create_binary_matrix(actualSets)
print("\nroute_matrix", route_matrix.shape, route_matrix[0])
# binary matrix where each row represents merchandise

merch_matrix = np.array([s[2] for s in actualSets])

print("\nmerch_matrix", merch_matrix.shape, merch_matrix)
print("merch_matrix contains nan", np.isnan(merch_matrix).any())

# compute Jaccard similarity for each matrix
route_similarity = jaccard_similarity_matrix(route_matrix)
print("\nroute_similarity", route_similarity.shape, route_similarity[0])
#merch_similarity = jaccard_similarity_matrix_merch(merch_matrix)
merch_similarity = similarity_matrix_merch(merch_matrix)
print("\nmerch_similarity", merch_similarity.shape, merch_similarity[0])

# compute final Jaccard distance
actualSetsDistances = (route_similarity + merch_similarity) / 2
actualSetsDistances = np.nan_to_num(actualSetsDistances, nan=0)
actualSetsDistances = 1 - actualSetsDistances
print("\nactualSetsDistances", actualSetsDistances.shape, actualSetsDistances[0])

uniqueShingles 870
shingle_to_index 870

route_matrix (10000, 870) [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0
 0 0 1 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0
 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 

In [31]:
# def jaccard_distance_routes(route1, route2):
#     id1 = route1[0]
#     id2 = route2[0]
#     r1 = set(route1[1])
#     r2 = set(route2[1])
#     merch1 = route1[2]
#     merch2 = route2[2]
    
#     intersection = len(list(r1.intersection(r2)))
#     union = (len(r1) + len(r2)) - intersection
#     jaccard_similarity = float(intersection) / union if union != 0 else 0
    
#     intersectionMerch = np.sum(np.minimum(merch1, merch2))
#     unionMerch = np.sum(np.maximum(merch1, merch2))
#     jaccard_similarity_merch = float(intersectionMerch) / unionMerch if unionMerch != 0 else 0
    
#     return 1 - (jaccard_similarity + jaccard_similarity_merch) / 2

# forward_expansion = len(actualSets) // len(standardSets)
# # precompute the distances between the elements of the actual sets
# actualSetsDistances = np.zeros((len(actualSets), len(actualSets)))
# for i in range(len(actualSets)):
#     for j in range(len(actualSets)):
#         actualSetsDistances[i,j] = jaccard_distance_routes(actualSets[i], actualSets[j])
#print("actualSetsDistances: ", actualSetsDistances.shape, actualSetsDistances[0])

In [32]:
# HDBSCAN clustering
# Compute HDBSCAN

# import gridsearchcv

#grid = GridSearchCV(hdbscan, scoring=hdbscan.probabilities_, , verbose=1)
# min_cluster_size = [10, 15, 20, 25, 30, 35, 40, 45, 50]
# max_cluster_size = [55, 60, 65, 75, 80, 85, 90, 95, 100]
forward_expansion = len(actualSets) // len(standardSets)

#from hdbscan import HDBSCAN as hdbsc

hdb = HDBSCAN(min_cluster_size=forward_expansion//3, max_cluster_size=forward_expansion, metric="precomputed", store_centers=None,allow_single_cluster=False ).fit(actualSetsDistances.copy())
#hdb = DBSCAN(eps=0.5, min_samples=40, metric="precomputed").fit(actualSetsDistances)

labels_HDBSCAN = hdb.labels_
print("num clusters found: ", len(set(labels_HDBSCAN)))
print("biggest cluster: ", max(labels_HDBSCAN, key=list(labels_HDBSCAN).count), " num elements: ", list(labels_HDBSCAN).count(max(labels_HDBSCAN, key=list(labels_HDBSCAN).count)))

# Create a color map that maps each unique label to a color
unique_labels = np.unique(labels_HDBSCAN)
#unique_labels = unique_labels[unique_labels != -1]
print("unique_labels: ", len(unique_labels), unique_labels, "standard sets len", len(standardSets))

#centroids = hdb.centroids_
#medoids = hdb.medoids_

#print("centroids: ", centroids.shape)
#print("medoids: ", medoids.shape)

# find the medoids using the clusters found by HDBSCAN
print("actualSetsDistances: ", actualSetsDistances.shape, actualSetsDistances[0])
medoidsIndices = []
cluster_mean_distances = []
for cluster in unique_labels:
    #print("cluster: ", cluster)
    if cluster in [-1, -2, -3]:
        continue
    cluster_elements = np.where(labels_HDBSCAN == cluster)[0]
    #print("cluster_elements: ", cluster_elements.shape, cluster_elements)
    #print("cluster_elements: ", actualSetsDistances[cluster_elements].shape)
    cluster_distances = actualSetsDistances[cluster_elements][:,cluster_elements]
    #print("cluster_distances: ", cluster_distances.shape, cluster_distances)
    #print("real distance", actualSetsDistances[cluster_elements[0], cluster_elements[0]])
    cluster_distances_sum = np.sum(cluster_distances, axis=1)
    cluster_distances_mean = np.mean(cluster_distances, axis=1)
    cluster_mean_distances.append(np.min(cluster_distances_mean))
    #print("cluster min mean distance: ", np.min(cluster_distances_mean))
    medoid = cluster_elements[np.argmin(cluster_distances_sum)]
    #medoidMean = cluster_elements[np.argmin(cluster_distances_mean)]
    #print("medoidMean", medoidMean, "medoid", medoid)
    #print("medoid", medoid)
    medoidsIndices.append(medoid)
medoidsIndices = np.array(medoidsIndices)

print("medoidsIndices: ", medoidsIndices.shape, medoidsIndices)
print("cluster_mean_distances: ", len(cluster_mean_distances), cluster_mean_distances)


num clusters found:  11
biggest cluster:  1  num elements:  1000
unique_labels:  11 [-1  0  1  2  3  4  5  6  7  8  9] standard sets len 10
actualSetsDistances:  (10000, 10000) [0.         0.1128246  0.15873655 ... 0.62467514 0.57965709 0.58009032]
medoidsIndices:  (10,) [9258  559 3263 2750 5587 4899 1017 6952 7143 8443]
cluster_mean_distances:  10 [0.08166804743846873, 0.12138224017205561, 0.11413835111791808, 0.12004603839790995, 0.1217398258173191, 0.11481063396427932, 0.11590109324457151, 0.11755424560114222, 0.11923819144546473, 0.1156510870077151]


In [33]:
completeSets = actualSets + standardSets
print("completeSets: ", len(completeSets))
route_matrix_with_standard = create_binary_matrix(completeSets)
# binary matrix where each row represents merchandise
merch_matrix_with_standard = np.array([s[2] for s in completeSets])

# compute Jaccard similarity for each matrix
route_similarity_with_standard = jaccard_similarity_matrix(route_matrix_with_standard)
merch_similarity_with_standard = similarity_matrix_merch(merch_matrix_with_standard)
completeSetsDistances = (route_similarity_with_standard + merch_similarity_with_standard) / 2
completeSetsDistances = np.nan_to_num(completeSetsDistances, nan=0)
completeSetsDistances = 1 - completeSetsDistances
print("completeSetsDistances", completeSetsDistances.shape, completeSetsDistances[0])

perplexity = 50 if len(completeSetsDistances) > 50 else len(completeSetsDistances) - 1
completeSetTSNE = TSNE(n_components=3, perplexity=perplexity, n_iter=1000, verbose=1).fit_transform(completeSetsDistances)

completeSets:  10010
uniqueShingles 870
shingle_to_index 870
matrix is not sparse
intersection (10010, 10010)
row_sums (10010,)
union (10010, 10010)
jaccard_similarity (10010, 10010)
jaccard_similarity contains nan False
matrix merch shape (10010, 10)
completeSetsDistances (10010, 10010) [0.         0.1128246  0.15873655 ... 0.52205851 0.53812329 0.58675415]
[t-SNE] Computing 151 nearest neighbors...
[t-SNE] Indexed 10010 samples in 0.019s...
[t-SNE] Computed neighbors for 10010 samples in 8.456s...
[t-SNE] Computed conditional probabilities for sample 1000 / 10010
[t-SNE] Computed conditional probabilities for sample 2000 / 10010
[t-SNE] Computed conditional probabilities for sample 3000 / 10010
[t-SNE] Computed conditional probabilities for sample 4000 / 10010
[t-SNE] Computed conditional probabilities for sample 5000 / 10010
[t-SNE] Computed conditional probabilities for sample 6000 / 10010
[t-SNE] Computed conditional probabilities for sample 7000 / 10010
[t-SNE] Computed condition

In [34]:


# reorder the labels to have colors matching the cluster results, using medoids which are closer to the standard vectors
medoidSets = [actualSets[i] for i in medoidsIndices]
print("medoidSets: ", len(medoidSets))

num_clusters_unique = unique_labels[unique_labels >= 0]
print("num_clusters_unique: ", len(num_clusters_unique), num_clusters_unique)

assert len(medoidSets) == len(num_clusters_unique), "The number of medoids is not equal to the number of unique labels"   

if len(medoidSets) == 0:
    print("No clustroids found")
else:

    route_matrix_standard, route_matrix_medoids = create_binary_matrices(standardSets, medoidSets)

    simMatrixMixed = jaccard_similarity_two_matrices(route_matrix_medoids, route_matrix_standard)

    print("route_matrix_standard: ", route_matrix_standard.shape)

    print("route_matrix_actual: ", route_matrix_medoids.shape)

    print("simMatrixMixed: ", simMatrixMixed.shape, simMatrixMixed[0])


    CAN_BE_ORDERED = False
    # get the closest standard vector for each medoid using simMatrixMixed

    argmax = np.argmax(simMatrixMixed, axis=1) # get the index of the closest standard vector for each medoid
    print("argmax: ", argmax.shape, argmax)

    if len(set(argmax)) == len(medoidsIndices): # if the argmax are all different, then the medoids can be reordered
        print("argmax is correct, can be reordered")
        CAN_BE_ORDERED = True
        # reorder medoidsIndices
        print("argmax: ", argmax.shape, argmax)
        print("medoidsIndices: ", medoidsIndices.shape, medoidsIndices)
        
        
        # Create an array of zeros with the same shape as the original array
        # result = np.zeros_like(medoidsIndices)
        # argsort = np.argsort(argmax)
        # print("argsort: ", argsort.shape, argsort)
        # # Fill the result array using the permutation indices
        # result[argsort] = medoidsIndices
        # medoidsIndicesReordered = result # reorder medoidsIndices
        # result = np.zeros_like(medoidsIndices)
        # unique_labels_filtered = unique_labels[unique_labels >= 0]
        # result[argsort] = unique_labels_filtered
        unique_labels_reordered = argmax   # 4, 2, 5, 10, ... -> 10, 4, 5, 2, ...
        
        #medoidsIndicesReordered = medoidsIndices[argmax] # reorder medoidsIndices
        #print("medoidsIndices: ", medoidsIndicesReordered.shape, medoidsIndicesReordered)
        #unique_labels = unique_labels[argmax]


    # for i in range(len(standardSets)):
    #     #distances = np.linalg.norm(medoids - standardVectors[i], axis=1)
    #     distancesCosine = []
    #     for j in range(len(medoidsIndices)):
    #         distancesCosine.append(cosine(actualSets[j], standardSets[i]))
    #     closest_medoid = np.argmin(distancesCosine)
        
    #     if closest_medoid not in unique_labels_reordered:
    #         unique_labels_reordered.append(closest_medoid)
    #     else:
    #         print("closest_medoid already in unique_labels_reordered")
    #         can_be_reordered = False
    #         break

    if not CAN_BE_ORDERED:
        #unique_labels_reordered = unique_labels

        #unique_labels = unique_labels_reordered
        #unique_labels = np.unique(labels_HDBSCAN)
        
        print("unique_labels: ", len(unique_labels), unique_labels)
    else:
        print("unique_labels_reordered: ", len(unique_labels_reordered), unique_labels_reordered)
        #unique_labels = unique_labels_reordered
        # compare if distances between clustroids and standard vectors are smaller than distances between standard vectors and other actual vectors

        #distancesClustroids = simMatrixMixed[np.arange(len(simMatrixMixed)), argmax]
        
    distancesClustroids = []
    distancesStandardVectors = []
    distancesStandardVectors2 = []
    # for i, clustroid in enumerate(medoidsIndices):
    #     distMedCluster = completeSetsDistances[:len(actualSets), :len(actualSets)][clustroid, labels_HDBSCAN == i]
    #     #print(labels_HDBSCAN)
    #     #print(labels_HDBSCAN == argmax[i])
    #     print("distMedCluster: ", np.sum(distMedCluster), len(distMedCluster), distMedCluster)
    #     # make distMedCluster a single list, no np.sum since they are inhomogeneous lists
    #     #print()
    #     #print(actualRefStandardIds)
    #     distStdCluster = completeSetsDistances[len(actualSets):, :len(actualSets)][i, np.where(np.array(actualRefStandardIds) == i)[0]]
    #     print("distStdCluster: ", np.sum(distStdCluster), len(distStdCluster), distStdCluster)
        
    #     distancesClustroids.append(np.mean(distMedCluster))
    #     distancesStandardVectors2.append(np.mean(distStdCluster))
    actualRefStandardIdsNumpy = np.array(actualRefStandardIds)
    for i, stdID in enumerate(standardRefIds):
        distStdCluster = completeSetsDistances[len(actualSets):, :len(actualSets)][i, np.where(actualRefStandardIdsNumpy == stdID)[0]]
        #print("distStdCluster: ", np.mean(distStdCluster), len(distStdCluster), distStdCluster)
        distancesStandardVectors.append(np.mean(distStdCluster))
    
    mean_distance_clustroids = np.mean(cluster_mean_distances)
    std_dev_distance_clustroids = np.std(cluster_mean_distances)

    mean_distance_standard_vectors = np.mean(distancesStandardVectors)
    std_dev_distance_standard_vectors = np.std(distancesStandardVectors)

    cv_clustroids = std_dev_distance_clustroids / mean_distance_clustroids
    cv_standard_vectors = std_dev_distance_standard_vectors / mean_distance_standard_vectors
    # print in green if the improvement is positive, in red if it is negative
    print("\n\033[94mMean distance from vectors of the same cluster to:")
    print("         clustroids: ", mean_distance_clustroids)
    #print("         first     : ", np.mean(distancesClustroids))
    print("   standard vectors: ", mean_distance_standard_vectors)
    #print("first                ", np.mean(distancesStandardVectors2))
    

    print("\nStd dev distance from vectors of the same cluster to:")
    print("         clustroids: ", std_dev_distance_clustroids)
    print("   standard vectors: ", std_dev_distance_standard_vectors)

    print("\nCoefficient of variation from vectors of the same cluster to:")
    print("         clustroids: ", cv_clustroids)
    print("   standard vectors: ", cv_standard_vectors)
    print("\033[0m")

    #ratio = np.mean(distancesClustroids) / np.mean(distancesStandardVectors)
    ratio = cv_clustroids / cv_standard_vectors
    percentage = (1 - ratio) * 100

    if percentage > 0:
        # print in green if the improvement is positive, in red if it is negative
        print("\n Improvement: \033[92m{:.2f}% \n\033[0m".format(percentage))
    else:
        print("\n Decline: \033[91m{:.2f}% \n\033[0m".format(-percentage))
    
    



# orderedClustroids = np.zeros_like(actualSetsDistances[medoidsIndices].copy())
# order = []
# for i, clustroid in enumerate(medoidsIndices):
#     distances = []
#     for j, standardVector in enumerate(standardSets):
#         distances.append(cosine(clustroid, standardVector))
        
#     distances = np.array(distances)
#     orderedClustroids[np.argmin(distances)] = clustroid
#     order.append(np.argmin(distances))
    
# print("order: ", order)

# if len(set(order)) == len(clustroids):
#     print("order is correct")
#     CAN_BE_ORDERED = True
#     clustroids = orderedClustroids
#     print("clustroids: ", clustroids.shape)
#     # reorder clustroidsIndices
#     clustroidsIndices = np.array(clustroidsIndices)[order]


#     # compare if distances between clustroids and standard vectors are smaller than distances between standard vectors and other standard vectors

#     distancesClustroids = []
#     distancesStandardVectors = []
#     for i, clustroid in enumerate(clustroids):
#         distancesClustroids.append(np.sum([cosine(clustroid, v) for v in actualClusters[i]]))
#         distancesStandardVectors.append(np.sum([cosine(standardVectors[i], v) for v in actualClusters[i]]))
        
#     print("distancesClustroids: ", distancesClustroids)
#     print("distancesStandardVectors: ", distancesStandardVectors)

#     print("\nMean distance from vectors of the same cluster to:")
#     print("         clustroids: ", np.mean(distancesClustroids))
#     print("   standard vectors: ", np.mean(distancesStandardVectors))

#     print("\nStd dev distance from vectors of the same cluster to:")
#     print("         clustroids: ", np.std(distancesClustroids))
#     print("   standard vectors: ", np.std(distancesStandardVectors))

#     print("\n improvement: ", 1 - np.mean(distancesClustroids) / np.mean(distancesStandardVectors), "\n")
# else:
#     print("NO ORDER IS POSSIBLE")
#     CAN_BE_ORDERED = False





medoidSets:  10
num_clusters_unique:  10 [0 1 2 3 4 5 6 7 8 9]
route_matrix_standard:  (10, 163)
route_matrix_actual:  (10, 163)
simMatrixMixed:  (10, 10) [0.         0.         0.         0.         0.         0.
 0.04166667 0.         0.         1.        ]
argmax:  (10,) [9 0 3 2 5 4 1 6 7 8]
argmax is correct, can be reordered
argmax:  (10,) [9 0 3 2 5 4 1 6 7 8]
medoidsIndices:  (10,) [9258  559 3263 2750 5587 4899 1017 6952 7143 8443]
unique_labels_reordered:  10 [9 0 3 2 5 4 1 6 7 8]

Mean distance from vectors of the same cluster to:
         clustroids:  0.11421297542068445
   standard vectors:  0.11484938091810074

Std dev distance from vectors of the same cluster to:
         clustroids:  0.011148750372989593
   standard vectors:  0.009319857935269502

Coefficient of variation from vectors of the same cluster to:
         clustroids:  0.09761369346980962
   standard vectors:  0.0811485256669821


 Decline: 20.29% 



In [40]:
max_len = max(len(standardSets), len(num_clusters_unique))
colors = plt.cm.jet(np.linspace(0, 1, max_len))
print("colors", colors.shape, colors)

print("medoid labels", [labels_HDBSCAN[i] for i in medoidsIndices])

if len(medoidSets) > 0 and CAN_BE_ORDERED:
    print("medoids added to plots and reordered")
    color_map = dict(zip(range(max_len), colors[unique_labels_reordered]))
    print("color_map", color_map)
else:
    color_map = dict(zip(range(max_len), colors))   # 0=red, 1=blue, 2=green, 3=yellow, 4=purple, 5=lightblue, 6=lightgreen, 7=lightyellow, 8=lightpurple
    
marker_colors = [color_map[label] if label > -1 else np.array([0,0,0,1]) for label in labels_HDBSCAN]
marker_colors_medoids = [color_map[label] if label > -1 else np.array([0,0,0,1]) for label in unique_labels]


# Create a trace for each type (centroids data)
traceStandard = go.Scatter3d(
    x=completeSetTSNE[len(actualSets):,0],
    y=completeSetTSNE[len(actualSets):,1],
    z=completeSetTSNE[len(actualSets):,2],
    mode='markers',
    marker=dict(
        size=7,
        color=colors,                # set color to an array/list of desired values
        opacity=1,
        symbol='diamond'
    )
)

if len(medoidSets) > 0:
    medoidsElements = completeSetTSNE[medoidsIndices]
    traceMedoids = go.Scatter3d(
        x=medoidsElements[:,0],
        y=medoidsElements[:,1],
        z=medoidsElements[:,2],
        mode='markers',
        marker=dict(
            size=7,
            color=marker_colors_medoids,                # set color to an array/list of desired values
            opacity=1,
            symbol='cross'
        )
    )

# Create a trace for each type (centroids data)
traceActual = go.Scatter3d(
    x=completeSetTSNE[:len(actualSets),0],
    y=completeSetTSNE[:len(actualSets),1],
    z=completeSetTSNE[:len(actualSets),2],
    mode='markers',
    marker=dict(
        size=7,
        color=marker_colors,                # set color to an array/list of desired values
        opacity=0.1,
        symbol='circle'
    )
)

# Plot
if len(medoidSets) > 0:
    data = [traceStandard, traceActual, traceMedoids]
else:
    data = [traceStandard, traceActual]

layout = go.Layout(
    margin=dict(
        l=0,
        r=0,
        b=0,
        t=0
    )
)

fig = go.Figure(data=data, layout=layout)
fig.show()

colors (10, 4) [[0.         0.         0.5        1.        ]
 [0.         0.         0.99910873 1.        ]
 [0.         0.37843137 1.         1.        ]
 [0.         0.83333333 1.         1.        ]
 [0.30044276 1.         0.66729918 1.        ]
 [0.66729918 1.         0.30044276 1.        ]
 [1.         0.90123457 0.         1.        ]
 [1.         0.48002905 0.         1.        ]
 [0.99910873 0.07334786 0.         1.        ]
 [0.5        0.         0.         1.        ]]
medoid labels [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
medoids added to plots and reordered
color_map {0: array([0.5, 0. , 0. , 1. ]), 1: array([0. , 0. , 0.5, 1. ]), 2: array([0.        , 0.83333333, 1.        , 1.        ]), 3: array([0.        , 0.37843137, 1.        , 1.        ]), 4: array([0.66729918, 1.        , 0.30044276, 1.        ]), 5: array([0.30044276, 1.        , 0.66729918, 1.        ]), 6: array([0.        , 0.        , 0.99910873, 1.        ]), 7: array([1.        , 0.90123457, 0.        , 1.        ]

## Ground Truth

In [36]:

colors_true = plt.cm.jet(np.linspace(0, 1, len(standardSets)))
color_map_true = dict(zip(range(max_len), colors))   # 0=red, 1=blue, 2=green, 3=yellow, 4=purple, 5=lightblue, 6=lightgreen, 7=lightyellow, 8=lightpurple
# marker colors for each point with the same color as the cluster it belongs to in the original data
marker_colors_true = [color_map_true[label] for label in actualRefStandardIds]


# Create a trace for each type (centroids data)
traceStandard_true = go.Scatter3d(
    x=completeSetTSNE[len(actualSets):,0],
    y=completeSetTSNE[len(actualSets):,1],
    z=completeSetTSNE[len(actualSets):,2],
    mode='markers',
    marker=dict(
        size=7,
        color=colors_true,                # set color to an array/list of desired values
        opacity=1,
        symbol='diamond'
    )
)

# Create a trace for each type (centroids data)
traceActual_true = go.Scatter3d(
    x=completeSetTSNE[:len(actualSets),0],
    y=completeSetTSNE[:len(actualSets),1],
    z=completeSetTSNE[:len(actualSets),2],
    mode='markers',
    marker=dict(
        size=7,
        color=marker_colors_true,                # set color to an array/list of desired values
        opacity=0.1,
        symbol='circle'
    )
)

# Plot
data = [traceStandard_true, traceActual_true]

layout = go.Layout(
    margin=dict(
        l=0,
        r=0,
        b=0,
        t=0
    )
)

fig = go.Figure(data=data, layout=layout)
fig.show()

## Output recommended standard routes

## t-SNE of centroids of clusters and actual routes

## t-SNE of clustroids, standard, and actual routes